# TODO:
- Live price
- Statistic for all cryptocurrencies
- Load the user data from exchanges
- Summary

In [18]:
import pandas
import requests
import json
import dotenv

dotenv.load_dotenv()

True

In [19]:
import os

# TODO: Implement
def get_current_price(crypto_ticker):
    url = 'https://rest.coinapi.io/v1/exchangerate/BTC/EUR'
    headers = {'X-CoinAPI-Key' : os.environ.get('COIN_API_KEY')}
    response = requests.get(url, headers=headers)
    response_dict = json.loads(response.text)
    return 40000


def get_summary_from_csv(df_buy, df_sell, column_keys, current_price):
    total_amout = df_buy[column_keys['size']].sum() - df_sell[column_keys['size']].sum()
    avg_price = df_buy[column_keys['price']].mean()

    invested_buy = df_buy[column_keys['total']].sum()
    invested = (-1 * invested_buy if invested_buy > 0 else invested_buy) - df_sell[column_keys['total']].sum()
    current_value = current_price * total_amout
    unrealized_profit = current_value + invested

    return { 'total_amount': total_amout, 'avg_price': avg_price, 'transaction_count': len(df_buy.axes[0]),
    'invested': invested, 'current_value': current_value, 'unrealized_profit': unrealized_profit }

def print_result(result):
    print(f"Total amout is {result['total_amount']}.")
    print(f"The average purchase price is {round(result['avg_price'])} EUR.")
    print(f"Total invested: {round(result['invested'])} EUR.")
    print(f"Portfolio value is {round(result['current_value'])} EUR.")
    print(f"Unrealized profit is {round(result['unrealized_profit'])} EUR.")

def merge_results(result1, result2):
    result = {}
    for key in result1.keys():
        if key == 'avg_price':
            result1_weight = result1[key] * result1['transaction_count']
            result2_weight = result2[key] * result2['transaction_count']
            result[key] = (result1_weight + result2_weight) / (result1['transaction_count'] + result2['transaction_count'])
        else:
            result[key] = result1[key] + result2[key]
    return result


In [20]:
# Coinbase
def get_coinbase_summary(current_price):
    df_coinbase = pandas.read_csv('data/btc-coinbase.csv')
    df_coinbase_buy = df_coinbase[df_coinbase['Transaction Type'] == 'Buy']
    df_coinbase_sell = df_coinbase[df_coinbase['Transaction Type'] == 'Sell']
    #print(df_coinbase_buy)
    return get_summary_from_csv(df_buy=df_coinbase_buy, df_sell=df_coinbase_sell, column_keys={ 'size': 'Quantity Transacted', 'price': 'Spot Price at Transaction', 'total': 'Total (inclusive of fees)'  }, current_price=current_price)

In [21]:
# Coinbase PRO
def get_coinbase_pro_summary(current_price):
    df = pandas.read_csv('data/btc-coinbase-pro.csv')
    df_buy = df[df['side'] == 'BUY']
    df_sell = df[df['side'] == 'SELL']
    return get_summary_from_csv(df_buy, df_sell, { 'size': 'size', 'price': 'price', 'total': 'total' }, current_price)


In [22]:
coinbase_pro_result = get_coinbase_pro_summary(get_current_price(''))
coinbase_result = get_coinbase_summary(get_current_price(''))

result = merge_results(coinbase_pro_result, coinbase_result)
print_result(result)


Total amout is 0.6785515.
The average purchase price is 26581 EUR.
Total invested: -10175 EUR.
Portfolio value is 27142 EUR.
Unrealized profit is 16967 EUR.
